
**Automated Detection of Violent Events in Video Streams** The project's goal is to develop and implement an advanced deep learning model capable of automatically recognizing violent activities in video streams, an importantconcern for public safety and surveillance systems. Leveraging the power ofconvolutional neural networks (CNNs) and/or recurrent neural networks (RNNs), the project combines spatial and temporal data analysis to accurately identify instancesof violence among individuals or groups within video footage.

**Data Collection:**

In [56]:
import numpy as np
import os
import cv2
from skimage.transform import resize
from tqdm.notebook import tqdm
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras import Input
from tqdm import tqdm
from tensorflow.keras.models import save_model
from keras.applications.inception_v3 import preprocess_input

In [151]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
violence = os.listdir('/content/drive/MyDrive/FinalProjectDeepLearning')
violence_path = [os.path.join('/content/drive/MyDrive/FinalProjectDeepLearning',name) for name in violence]

In [46]:
violence_path[1]

'/content/drive/MyDrive/FinalProjectDeepLearning/Violence_detection (1).ipynb'

**Preprocessing Of Data:**

In [149]:
# Exemple d'utilisation
video_frames = preprocess_video('Violence_Video.mp4', frame_interval=5, target_size=(224, 224))
augmented_frames = data_augmentation(video_frames)

In [48]:
def preprocess_video(video_path, frame_interval=1, target_size=(224, 224)):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # Shrink frame rate based on specified interval
        if cap.get(cv2.CAP_PROP_POS_FRAMES) % frame_interval == 0:
            # Resizing the frame to the target size
            frame = cv2.resize(frame, target_size)
            frames.append(frame)
    cap.release()
    return frames

def data_augmentation(frames):
    augmented_frames = []
    for frame in frames:
        # Example of transformation: horizontal flip
        flipped_frame = cv2.flip(frame, 1)
        augmented_frames.append(flipped_frame)
    return augmented_frames

In [49]:
# Exemple d'utilisation
video_frames = preprocess_video('Violence_Video.mp4', frame_interval=5, target_size=(224, 224))
augmented_frames = data_augmentation(video_frames)

**Model Designing and Implementation:**

In [50]:
pretrained_model = InceptionV3()
# Create a new model for feature extraction
# Extract features from the second-to-last layer of the InceptionV3 model
pretrained_model = Model(inputs=pretrained_model.input,outputs=pretrained_model.layers[-2].output)
pretrained_model.summary()

96112376/96112376 [==============================] - 1s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 149, 149, 32)         864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 149, 149, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     

In [52]:
def feature_extractor(frame):
    # Preprocess the frame for compatibility with the pre-trained model
    img = preprocess_input(frame)

    # Expand the dimensions of the frame for model compatibility
    img = np.expand_dims(img, axis=0)

    # Use the pre-trained feature extraction model to obtain the feature vector
    feature_vector = pretrained_model.predict(img)

    # Flatten the feature vector if needed
    feature_vector = feature_vector.flatten()

    # Return the extracted feature vector
    return feature_vector

**Video Frames Extraction Function:**

In [101]:
def frames_extraction(video_path, SEQUENCE_LENGTH=16, IMAGE_WIDTH=299, IMAGE_HEIGHT=299, total_video=None):
    # Your function implementation here):
    # List to store features for all videos
    all_video_features = []

    # Loop through each video
    for video_file in tqdm(video_path):
        frames_list = []

        # Open the video file for reading
        video_reader = cv2.VideoCapture(video_file)

        # Get the total number of frames in the video
        video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))

        # Calculate the number of frames to skip in order to achieve the desired sequence length
        skip_frames_window = max(int(video_frames_count / SEQUENCE_LENGTH), 1)

        # Read the first frame to calculate the feature vector length
        success, frame = video_reader.read()
        if not success:
            continue  # Skip to the next video if unable to read the frame

        # Convert the frame to RGB and resize it
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        resized_frame = cv2.resize(frame_rgb, (IMAGE_HEIGHT, IMAGE_WIDTH))

        # Normalize the frame
        normalized_frame = resized_frame / 255

        # Extract features using the previously defined feature extraction function (assuming it's defined)
        features = feature_extractor(normalized_frame)
        FEATURE_VECTOR_LENGTH = len(features)

        # Loop through each frame in the sequence
        for frame_counter in range(SEQUENCE_LENGTH):
            # Set the position of the video reader to the current frame
            video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)

            # Read the frame
            success, frame = video_reader.read()

            # Break if unable to read the frame
            if not success:
                break

            # Convert the frame to RGB and resize it
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            resized_frame = cv2.resize(frame_rgb, (IMAGE_HEIGHT, IMAGE_WIDTH))

            # Normalize the frame
            normalized_frame = resized_frame / 255

            # Extract features using the previously defined feature extraction function (assuming it's defined)
            features = feature_extractor(normalized_frame)

            # Append the features to the list
            frames_list.append(features)

        # Pad or truncate the frames_list to have a fixed length
        if len(frames_list) == 0:
            frames_list = [np.zeros((FEATURE_VECTOR_LENGTH,))] * SEQUENCE_LENGTH
        elif len(frames_list) < SEQUENCE_LENGTH:
            frames_list += [np.zeros_like(frames_list[0])] * (SEQUENCE_LENGTH - len(frames_list))
        elif len(frames_list) > SEQUENCE_LENGTH:
            frames_list = frames_list[:SEQUENCE_LENGTH]

        # Append the list of features for the current video to the overall list
        all_video_features.append(frames_list)

        # Release the video reader
        video_reader.release()

    # Convert the list of features to a numpy array
    return np.array(all_video_features)


In [152]:
violence_features = frames_extraction(violence_path[:500])

Error extracting features: Failed to open video file: Violence_Video.mp4


In [97]:
np.save('/content/drive/MyDrive/FinalProjectDeepLearning/violence_features.npy',violence_features)# save the feature in our directory and make it reusable

In [98]:
violence_data = np.load('/content/drive/MyDrive/FinalProjectDeepLearning/violence_features.npy')

In [66]:
violence_data[0].shape

(16, 2048)

**Creating LSTM Model and Preparing Data¶ In this section, we define an Bidirectional LSTM (Long Short-Term Memory) model for video classification and prepare the data for training.**

In [70]:
from keras.models import Sequential
from keras.layers import LSTM, Dense,Bidirectional,BatchNormalization,Dropout
from sklearn.model_selection import train_test_split
import numpy as np

# Create labels
violence_labels = np.zeros(len(violence_data))

# Combine features and labels
X = np.concatenate([violence_data], axis=0)
y = np.concatenate([violence_labels], axis=0)

In [71]:
len(X)# total samples

1

In [72]:
X[0].shape# shape of each samples

(16, 2048)

In [73]:
y[0:20]# first 20 labels

array([0.])

In [84]:
import numpy as np
from sklearn.model_selection import train_test_split

# Generate synthetic feature vectors
num_samples = 100  # Increase the number of samples
num_frames = 16
feature_vector_length = 2048
synthetic_features = np.random.rand(num_samples, num_frames, feature_vector_length)

# Generate synthetic labels (assuming binary classification: 0 for non-violence, 1 for violence)
synthetic_labels = np.random.randint(0, 2, size=num_samples)

# Check the shapes of the synthetic data
print("Synthetic Features Shape:", synthetic_features.shape)
print("Synthetic Labels Shape:", synthetic_labels.shape)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(synthetic_features, synthetic_labels, test_size=0.2, random_state=32)

X_train_reshaped = X_train.reshape((X_train.shape[0], 16, 2048))  # reshape to (16,2048)
X_test_reshaped = X_test.reshape((X_test.shape[0], 16, 2048))  # reshape to (16,2048)


Synthetic Features Shape: (100, 16, 2048)
Synthetic Labels Shape: (100,)


**LSTM Model Definition using Keras Functional API**

In [89]:
from tensorflow.keras.layers import Input, LSTM, Bidirectional, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model

# Define the input layer
inputs = Input(shape=(16, 2048))

# Build the LSTM model using Functional API
x = Bidirectional(LSTM(128, return_sequences=True))(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Bidirectional(LSTM(64))(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu')(x)
outputs = Dense(1, activation='sigmoid')(x)

# Create the model
model = Model(inputs=inputs, outputs=outputs)


In [90]:
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 16, 2048)]        0         
                                                                 
 bidirectional_4 (Bidirecti  (None, 16, 256)           2229248   
 onal)                                                           
                                                                 
 batch_normalization_98 (Ba  (None, 16, 256)           1024      
 tchNormalization)                                               
                                                                 
 dropout_4 (Dropout)         (None, 16, 256)           0         
                                                                 
 bidirectional_5 (Bidirecti  (None, 128)               164352    
 onal)                                                           
                                                           

**Compiling The Model**

In [91]:
# Compile your model with an appropriate loss and optimizer
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_reshaped,y_train,validation_data=(X_test_reshaped,y_test),epochs=5,batch_size=32)

Epoch 1/5
3/3 [==============================] - 11s 1000ms/step - loss: 1.0924 - accuracy: 0.3875 - val_loss: 0.6857 - val_accuracy: 0.6000
Epoch 2/5
3/3 [==============================] - 1s 189ms/step - loss: 0.9028 - accuracy: 0.4625 - val_loss: 0.6936 - val_accuracy: 0.5500
Epoch 3/5
3/3 [==============================] - 1s 186ms/step - loss: 0.7921 - accuracy: 0.6125 - val_loss: 0.6904 - val_accuracy: 0.6500
Epoch 4/5
3/3 [==============================] - 1s 189ms/step - loss: 0.7142 - accuracy: 0.6750 - val_loss: 0.6916 - val_accuracy: 0.5500
Epoch 5/5
3/3 [==============================] - 1s 181ms/step - loss: 0.5724 - accuracy: 0.6875 - val_loss: 0.6864 - val_accuracy: 0.6000


**Model Evaluation**

In [134]:
# Evaluate the model on the test set
accuracy = model.evaluate(X_test_reshaped, y_test)
print("Test Accuracy:", accuracy[1])

1/1 [==============================] - 0s 65ms/step - loss: 0.6864 - accuracy: 0.6000
Test Accuracy: 0.6000000238418579


**Classification Report For The Model Prediction**

In [145]:
from sklearn.metrics import classification_report

# Predict the labels for the test set
y_pred = model.predict(X_test_reshaped)

# Convert the predicted probabilities to binary labels
y_preds = [1 if pred > 0.5 else 0 for pred in y_pred]

# Generate a classification report
report = classification_report(y_test, y_preds)

# Print the classification report
print("Classification Report:\n", report)

1/1 [==============================] - 0s 112ms/step
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.60      1.00      0.75        12

    accuracy                           0.60        20
   macro avg       0.30      0.50      0.37        20
weighted avg       0.36      0.60      0.45        20



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
